# Description

Runs S-PrediXcan on a set of traits.

# Modules

In [1]:
import os
from glob import glob
from pathlib import Path

from utils import read_log_file_and_check_line_exists
import conf

# Settings

Apparently, there is no easy way to get the parent directory of
a notebook in Jupyter, so here I get that information either from
the parameter sent by `nbs/run_nbs.sh` (if called from command-line) or
from `os.getcwd()` (if called from browser).

In [2]:
PHENOPLIER_NOTEBOOK_FILEPATH = None
PHENOPLIER_NOTEBOOK_DIR = os.getcwd()

PROJECTS_TRAIT_KEY = "CHRONOTYPE"

In [3]:
# Parameters
PHENOPLIER_NOTEBOOK_FILEPATH = "projects/chronotype/nbs/15_twas/01-run_spredixcan.ipynb"

In [4]:
if PHENOPLIER_NOTEBOOK_FILEPATH is not None:
    PHENOPLIER_NOTEBOOK_DIR = str(Path(PHENOPLIER_NOTEBOOK_FILEPATH).parent)

display(PHENOPLIER_NOTEBOOK_DIR)

'projects/chronotype/nbs/15_twas'

In [5]:
OUTPUT_DIR = conf.PROJECTS[PROJECTS_TRAIT_KEY]["RESULTS_DIR"] / "twas" / "spredixcan"
display(OUTPUT_DIR)

OUTPUT_DIR_STR = str(OUTPUT_DIR)
display(OUTPUT_DIR_STR)

PosixPath('/opt/data/projects/chronotype/results/twas/spredixcan')

'/opt/data/projects/chronotype/results/twas/spredixcan'

# Run

In [6]:
%%bash -s "$PHENOPLIER_NOTEBOOK_DIR" "$PROJECTS_TRAIT_KEY" "$OUTPUT_DIR_STR"
set -euo pipefail
# IFS=$'\n\t'

# read the notebook directory parameter and remove $1
export PHENOPLIER_NOTEBOOK_DIR="${PHENOPLIER_CODE_DIR}/$1"
shift

# read trait key
export PROJECTS_TRAIT_KEY="$1"
shift

# read output dir
export OUTPUT_DIR="$1"
shift

run_job () {
    # run_job is a standard function name that performs a particular job
    # depending on the context. It will be called by GNU Parallel below.
    #
    # The implementation here runs the GLS model of PhenoPLIER on a trait.

    # read trait information
    # the first parameter to this function is a string with values separated by
    # commas (,). So here I split those into different variables.
    IFS=',' read -r pheno_id file sample_size n_cases tissue <<< "$1"

    INPUT_FILENAME=${file%.*}

    # get input GWAS file, there should be a single file
    # here I make sure that there are no other files in the folder that
    # match this phenotype/trait filename prefix
    env_name="PHENOPLIER_PROJECTS_${PROJECTS_TRAIT_KEY}_RESULTS_DIR"
    GWAS_DIR="${!env_name}/final_imputed_gwas"
    N_GWAS_FILES=$(ls ${GWAS_DIR}/${INPUT_FILENAME}*.txt.gz | wc -l)
    if [ "${N_GWAS_FILES}" != "1" ]; then
        echo "ERROR: found ${N_GWAS_FILES} GWAS files instead of one"
        exit 1
    fi
    INPUT_GWAS_FILEPATH=$(ls ${GWAS_DIR}/${INPUT_FILENAME}*.txt.gz)

    mkdir -p "${OUTPUT_DIR}"

    # make sure we are not also parallelizing within numpy, etc
    export NUMBA_NUM_THREADS=1
    export MKL_NUM_THREADS=1
    export OPEN_BLAS_NUM_THREADS=1
    export NUMEXPR_NUM_THREADS=1
    export OMP_NUM_THREADS=1

    echo "Running for $pheno_id, $tissue"
    echo "Saving results in ${OUTPUT_DIR}"

    bash "${PHENOPLIER_CODE_DIR}/scripts/twas_spredixcan.sh" \
        --input-gwas-file "${INPUT_GWAS_FILEPATH}" \
        --phenotype-name "${INPUT_FILENAME}" \
        --tissue "${tissue}" \
        --output-dir "${OUTPUT_DIR}"

    # print errors here in the notebook
    # first, look for the log file for this trait
    pattern="${OUTPUT_DIR}/${INPUT_FILENAME}-gtex_v8-mashr-${tissue}*.log"

    N_LOG_FILES=$(ls ${pattern} | wc -l)
    if [ "${N_LOG_FILES}" != "1" ]; then
        echo "ERROR: found ${N_LOG_FILES} log files instead of one"
        exit 1
    fi
    LOG_FILE=$(ls ${pattern})

    cat "${LOG_FILE}" | grep -iE "warning|error"

    echo
}

# export function so GNU Parallel can see it
export -f run_job

# generate a list of run_job calls for GNU Parallel
# here I read a file with information about traits (one trait per line)
env_name="PHENOPLIER_PROJECTS_${PROJECTS_TRAIT_KEY}_TRAITS_INFO_FILE"
while IFS= read -r line; do
    for tissue in ${PHENOPLIER_PHENOMEXCAN_PREDICTION_MODELS_MASHR_TISSUES}; do
        echo run_job "${line},${tissue}"
    done
done < <(tail -n "+2" "${!env_name}") |
    parallel -k --lb --halt 2 -j${PHENOPLIER_GENERAL_N_JOBS}

Running for chronotype, Thyroid
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan


+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SPrediXcan.py --model_db_path /opt/data/data/phenomexcan/prediction_models/mashr/mashr_Thyroid.db --covariance /opt/data/data/phenomexcan/prediction_models/mashr/mashr_Thyroid.txt.gz --gwas_file /opt/data/projects/chronotype/results/final_imputed_gwas/chronotype_raw_BOLT.output_HRC.only_plus.metrics_maf0.001_hwep1em12_info0.3.txt-harmonized-imputed.txt.gz --separator '	' --non_effect_allele_column non_effect_allele --effect_allele_column effect_allele --snp_column panel_variant_id --zscore_column zscore --keep_non_rsid --additional_output --model_db_snp_key varID --throw --output_file /opt/data/projects/chronotype/results/twas/spredixcan/chronotype_raw_BOLT.output_HRC.only_plus.metrics_maf0.001_hwep1em12_info0.3.txt-gtex_v8-mashr-Thyroid.csv


WARNING - Some GWAS snp zscores are not finite.



+ set +x


Running for chronotype, Artery_Aorta
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.

Running for chronotype, Heart_Atrial_Appendage
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.

Running for chronotype, Liver
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.



...

+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SPrediXcan.py --model_db_path /opt/data/data/phenomexcan/prediction_models/mashr/mashr_Adrenal_Gland.db --covariance /opt/data/data/phenomexcan/prediction_models/mashr/mashr_Adrenal_Gland.txt.gz --gwas_file /opt/data/projects/chronotype/results/final_imputed_gwas/chronotype_raw_BOLT.output_HRC.only_plus.metrics_maf0.001_hwep1em12_info0.3.txt-harmonized-imputed.txt.gz --separator '	' --non_effect_allele_column non_effect_allele --effect_allele_column effect_allele --snp_column panel_variant_id --zscore_column zscore --keep_non_rsid --additional_output --model_db_snp_key varID --throw --output_file /opt/data/projects/chronotype/results/twas/spredixcan/chronotype_raw_BOLT.output_HRC.only_plus.metrics_maf0.001_hwep1em12_info0.3.txt-gtex_v8-mashr-Adrenal_Gland.csv
+ set +x
+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SPrediXcan.py --model_db_path /opt/data/dat

WARNING - Some GWAS snp zscores are not finite.



+ set +x


Running for chronotype, Ovary
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.

Running for chronotype, Cells_Cultured_fibroblasts
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.

Running for chronotype, Pituitary
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.

Running for chronotype, Kidney_Cortex
Saving results in /opt/data/projects/chronotype/results/twas/spredixcan
WARNING - Some GWAS snp zscores are not finite.



+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SPrediXcan.py --model_db_path /opt/data/data/phenomexcan/prediction_models/mashr/mashr_Ovary.db --covariance /opt/data/data/phenomexcan/prediction_models/mashr/mashr_Ovary.txt.gz --gwas_file /opt/data/projects/chronotype/results/final_imputed_gwas/chronotype_raw_BOLT.output_HRC.only_plus.metrics_maf0.001_hwep1em12_info0.3.txt-harmonized-imputed.txt.gz --separator '	' --non_effect_allele_column non_effect_allele --effect_allele_column effect_allele --snp_column panel_variant_id --zscore_column zscore --keep_non_rsid --additional_output --model_db_snp_key varID --throw --output_file /opt/data/projects/chronotype/results/twas/spredixcan/chronotype_raw_BOLT.output_HRC.only_plus.metrics_maf0.001_hwep1em12_info0.3.txt-gtex_v8-mashr-Ovary.csv
+ set +x
+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SPrediXcan.py --model_db_path /opt/data/data/phenomexcan/prediction

# Perform some checks in output and log files

In [7]:
assert OUTPUT_DIR.exists()

In [8]:
log_files = OUTPUT_DIR.glob("*.log")

In [9]:
for f in log_files:
    read_log_file_and_check_line_exists(
        f,
        [
            "INFO - 90 % of model's snps found",
            "INFO - Sucessfully processed metaxcan association",
        ],
    )